In [ ]:
!pip install ultralytics
!pip install EasyOCR

In [ ]:
NGROK_URL = "https://guadalupe-unstudied-honeyedly.ngrok-free.dev/plates"
API_KEY = "MY_SECRET_KEY_123"
LOCATION_NAME = "Gate A"
LATITUDE = 30.354
LONGITUDE = 76.37

MODEL_PATH = "/content/license_plate_best.pt"     # YOLO model
INPUT_VIDEO = "/content/demo_video.mp4"           # input video
OUTPUT_VIDEO = "/content/output_video.mp4"        # output video path

CONF_THRESH = 0.3


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import easyocr
import re
import requests
from datetime import datetime, UTC
import csv
from collections import defaultdict

model = YOLO(MODEL_PATH)
reader = easyocr.Reader(['en'], gpu=True)

pattern = re.compile(r'^[A-Z][0-9]{3}[A-Z]{2}$')

SERVER_URL = NGROK_URL
API_KEY_HEADER = API_KEY

plate_counts = defaultdict(int)
final_plates = set()

def send_to_server(plate):
    payload = {
        "plate": plate,
        "timestamp": datetime.now(UTC).isoformat(),
        "location": LOCATION_NAME,
        "latitude": LATITUDE,
        "longitude": LONGITUDE
    }
    headers = {"X-API-KEY": API_KEY_HEADER}
    try:
        resp = requests.post(SERVER_URL, json=payload, headers=headers, timeout=10)
        if resp.status_code == 201:
            print("Sent: True")
        else:
            print("Sent: False")
    except:
        print("Sent: False")


cap = cv2.VideoCapture(INPUT_VIDEO)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(
    OUTPUT_VIDEO,
    fourcc,
    cap.get(cv2.CAP_PROP_FPS),
    (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
)

csv_file = open("detected_plates.csv", "w", newline="")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["timestamp", "plate"])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)

    for r in results:
        for box in r.boxes:
            try:
                conf = float(box.conf.cpu().numpy().item())
            except:
                try:
                    conf = float(box.conf)
                except:
                    continue

            if conf < CONF_THRESH:
                continue

            try:
                x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy()[0])
            except:
                x1, y1, x2, y2 = map(int, box.xyxy.tolist()[0])

            h, w = frame.shape[:2]
            x1 = max(0, min(x1, w - 1))
            x2 = max(0, min(x2, w - 1))
            y1 = max(0, min(y1, h - 1))
            y2 = max(0, min(y2, h - 1))
            if x2 <= x1 or y2 <= y1:
                continue

            crop = frame[y1:y2, x1:x2]
            if crop is None or crop.size == 0:
                continue

            ocr = reader.readtext(crop, detail=0)
            plate = ocr[0].replace(" ", "") if ocr else ""
            if not pattern.match(plate):
                continue

            plate_counts[plate] += 1

            if plate_counts[plate] >= 4 and plate not in final_plates:
                final_plates.add(plate)
                csv_writer.writerow([datetime.now(UTC).isoformat(), plate])
                send_to_server(plate)

            if plate in final_plates:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                cv2.putText(frame, plate, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    out.write(frame)

cap.release()
out.release()
csv_file.close()

print("Saved CSV: detected_plates.csv")
print("Final Plates:", list(final_plates))


Sent: True
Sent: True
Sent: True
Sent: True
Saved CSV: detected_plates.csv
Final Plates: ['K884RS', 'L605HZ', 'L656XH', 'H644LX']
